In [2]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html
import numpy as np
import lxml

In [80]:
# Accedemos al contenido de cada página (elecciones 2021 y 2019) de cada municipio (desde el 01 al 173)
count_muni = 1
list_muni = []
lista_fallos = []
while count_muni <184:
    try:
        # coletilla
        if len(str(count_muni)) == 1:
            muniID = '0' + str(count_muni)
        else:
            muniID = str(count_muni)
        col =  muniID+'.xml2'
        
        # Se crea la lista donde se guardarán los datos de cada municipio 
        datos_muni = []

        # Se generan los accesos a las páginas web donde se alojan los datos (una versión para las elecciones de 2019 y otra para las de 2021)
        # 2021
        URL_2021 = 'https://rsl00.epimg.net/elecciones/2021/autonomicas/12/28/' + col
        r_2021 = requests.get (URL_2021)
        soup_2021 = BeautifulSoup(r_2021.text, 'lxml')
        #2019
        URL_2019 = 'https://rsl00.epimg.net/elecciones/2019/autonomicas/12/28/' + col
        r_2019 = requests.get (URL_2019)
        soup_2019 = BeautifulSoup(r_2019.text, 'lxml')

        # Se obtienen e insertan los datos de cabecera de cada municipio: 
        # ID, Nombre Municipio, Votos contabilizados, nulos y abstenciones, por año y con la variación entre elecciones (Delta) 
        # se inserta el código postal del municipio creado a partir del contador del bucle
        if len(str(muniID)) == 2:
            CPmuni = '280' + str(muniID)
        else:
            CPmuni = '28' + str(muniID)
        datos_muni.append(int(CPmuni))
        datos_muni.append(soup_2021.find('nombre_sitio').text)


        #Contabilizados
        conta_2021 =  float(soup_2021.find('votos').find('contabilizados').find('porcentaje').text)
        datos_muni.append(conta_2021)
        conta_2019 =  float(soup_2019.find('votos').find('contabilizados').find('porcentaje').text)
        datos_muni.append(conta_2019)
        delta_conta = round(conta_2021 -conta_2019, 2)
        datos_muni.append(delta_conta)

        # Abstenciones
        abst_2021 =  float(soup_2021.find('votos').find('abstenciones').find('porcentaje').text)
        datos_muni.append(abst_2021)
        abst_2019 =  float(soup_2019.find('votos').find('abstenciones').find('porcentaje').text)
        datos_muni.append(abst_2019)
        delta_abst = round(abst_2021 -abst_2019, 2)
        datos_muni.append(delta_abst)

        #Nulos
        nulos_2021 =  float(soup_2021.find('votos').find('nulos').find('porcentaje').text)
        datos_muni.append(nulos_2021)
        nulos_2019 =  float(soup_2019.find('votos').find('nulos').find('porcentaje').text)
        datos_muni.append(nulos_2019)
        delta_nulos = round(nulos_2021 -nulos_2019, 2)
        datos_muni.append(delta_nulos)

        # Blancos
        blc_2021 =  float(soup_2021.find('votos').find('blancos').find('porcentaje').text)
        datos_muni.append(blc_2021)
        blc_2019 =  float(soup_2019.find('votos').find('blancos').find('porcentaje').text)
        datos_muni.append(blc_2019)
        delta_blc = round(blc_2021 -blc_2019, 2)
        datos_muni.append(delta_blc)

        # Se genera el cuerpo central de la línea del municipio:
        # con los valores de los votos (y los deltas) de los partidos más relevantes: PP, PSOE, MÁS MADRID, PODEMOS-IU, VOX y Cs
        # Para ello se crean dos diccionarios auxiliares que filtran aquellos partidos que no son interesantes y servirán de base para generar 
        # un único diccionario con los datos de los partidos políticos relevantes del municipio
        partidos_2021 = soup_2021.find('resultados').find_all('partido') 
        dict_Municipio_2021 = {}
        for partido in partidos_2021:
            nombre_partido = partido.find('nombre').text
            if nombre_partido in ['PP','PSOE', 'MÁS MADRID', 'PODEMOS-IU', 'VOX', 'Cs']:
                porcentaje_partido = float(partido.find('votos_porciento').text)
                dict_Municipio_2021[nombre_partido] = porcentaje_partido
        dict_Municipio_2021

        partidos_2019 = soup_2019.find('resultados').find_all('partido') 
        dict_Municipio_2019 = {}
        for partido in partidos_2019:
            nombre_partido = partido.find('nombre').text
            if nombre_partido in ['PP','PSOE', 'MÁS MADRID', 'PODEMOS-IU', 'VOX', 'Cs']:
                porcentaje_partido = float(partido.find('votos_porciento').text)
                dict_Municipio_2019[nombre_partido] = porcentaje_partido
        dict_Municipio_2019

        # Después de filtrar los partidos relevantes se genera un diccionario con los votos de ambas elecciones y el Delta entre ellas
        dict_partidos = {}
        for key in dict_Municipio_2021:
            try:
                dict_partidos[key] = [dict_Municipio_2021[key], dict_Municipio_2019[key], round(dict_Municipio_2021[key] - dict_Municipio_2019[key],2)]
            except:
                continue

        # Se insertan los datos de los partidos siempre en el mismo orden 
        # PP: votos 2021, votos 2019, Delta
        try:
            datos_muni.append(dict_partidos['PP'][0])
            datos_muni.append(dict_partidos['PP'][1])
            datos_muni.append(dict_partidos['PP'][2])
        except:
            datos_muni.append(0)
            datos_muni.append(0)
            datos_muni.append(0)

        # PSOE: votos 2021, votos 2019, Delta
        try:
            datos_muni.append(dict_partidos['PSOE'][0])
            datos_muni.append(dict_partidos['PSOE'][1])
            datos_muni.append(dict_partidos['PSOE'][2])
        except:
            datos_muni.append(0)
            datos_muni.append(0)
            datos_muni.append(0)

        # MÁS MADRID: votos 2021, votos 2019, Delta
        try:
            datos_muni.append(dict_partidos['MÁS MADRID'][0])
            datos_muni.append(dict_partidos['MÁS MADRID'][1])
            datos_muni.append(dict_partidos['MÁS MADRID'][2])
        except:
            datos_muni.append(0)
            datos_muni.append(0)
            datos_muni.append(0)

        # PODEMOS-IU: votos 2021, votos 2019, Delta
        try:
            datos_muni.append(dict_partidos['PODEMOS-IU'][0])
            datos_muni.append(dict_partidos['PODEMOS-IU'][1])
            datos_muni.append(dict_partidos['PODEMOS-IU'][2])
        except:
            datos_muni.append(0)
            datos_muni.append(0)
            datos_muni.append(0)

        # VOX: votos 2021, votos 2019, Delta
        try:
            datos_muni.append(dict_partidos['VOX'][0])
            datos_muni.append(dict_partidos['VOX'][1])
            datos_muni.append(dict_partidos['VOX'][2])
        except:
            datos_muni.append(0)
            datos_muni.append(0)
            datos_muni.append(0)


        # Cs: votos 2021, votos 2019, Delta
        try:
            datos_muni.append(dict_partidos['Cs'][0])
            datos_muni.append(dict_partidos['Cs'][1])
            datos_muni.append(dict_partidos['Cs'][2])
        except:
            datos_muni.append(0)
            datos_muni.append(0)
            datos_muni.append(0)

        # se meten los datos en una lista
        list_muni.append(datos_muni)

        count_muni += 1
    except:
        lista_fallos.append(count_muni)
        count_muni += 1

# Se genera el dataframe con todos los datos recolectados así como las columnas que corresponden
df_votos_muni = pd.DataFrame(list_muni, columns=['CP', 'Municipio', 'Cont-2021', 'Cont-2019','D-Cont', 'Abst-2021', 'Abst-2019','D-Abst', 'Blc-2021', 'Blc-2019','D-Blc', 'Nul-2021', 'Nul-2019','D-Nul', 'PP-2021', 'PP-2019', 'D-PP', 'PSOE-2021', 'PSOE-2019', 'D-PSOE', 'MÁS MADRID-2021', 'MÁS MADRID-2019', 'D-MÁS MADRID', 'PODEMOS-IU-2021', 'PODEMOS-IU-2019', 'D-PODEMOS-IU', 'VOX-2021', 'VOX-2019', 'D-VOX', 'Cs-2021', 'Cs-2019', 'D-Cs'])
df_votos_muni.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CP               176 non-null    int64  
 1   Municipio        176 non-null    object 
 2   Cont-2021        176 non-null    float64
 3   Cont-2019        176 non-null    float64
 4   D-Cont           176 non-null    float64
 5   Abst-2021        176 non-null    float64
 6   Abst-2019        176 non-null    float64
 7   D-Abst           176 non-null    float64
 8   Blc-2021         176 non-null    float64
 9   Blc-2019         176 non-null    float64
 10  D-Blc            176 non-null    float64
 11  Nul-2021         176 non-null    float64
 12  Nul-2019         176 non-null    float64
 13  D-Nul            176 non-null    float64
 14  PP-2021          176 non-null    float64
 15  PP-2019          176 non-null    float64
 16  D-PP             176 non-null    float64
 17  PSOE-2021       

In [79]:
# se obtiene la población de los ficheros descargados del INE (población total y por sexos) para generar un único dataframe e insertarlo en el dataframe con los datos de votación

# Procesado del fichero con la población total
df_pop2021_T = pd.read_csv('pob_muni_INE2021.csv', sep=';')
df_pop2021_T['CP'] = df_pop2021_T['Municipios'].apply(lambda x: x.split()[0])
df_pop2021_T.drop(index=0, inplace=True)
df_pop2021_T.drop(columns=['Municipios','Sexo','Periodo'], inplace=True)
df_pop2021_T.rename(columns = {'Total':'Tot_Pop_2020'}, inplace=True)

# Procesado del fichero con la población total
df_pop2021_H = pd.read_csv('pob_muni_INE2021_H.csv', sep=';')
df_pop2021_H['CP'] = df_pop2021_H['Municipios'].apply(lambda x: x.split()[0])
df_pop2021_H.drop(index=0, inplace=True)
df_pop2021_H.drop(columns=['Municipios','Sexo','Periodo'], inplace=True)
df_pop2021_H.rename(columns = {'Total':'Pop_H_2020'}, inplace=True)

# Procesado del fichero con la población total
df_pop2021_M = pd.read_csv('pob_muni_INE2021_M.csv', sep=';')
df_pop2021_M['CP'] = df_pop2021_M['Municipios'].apply(lambda x: x.split()[0])
df_pop2021_M.drop(index=0, inplace=True)
df_pop2021_M.drop(columns=['Municipios','Sexo','Periodo'], inplace=True)
df_pop2021_M.rename(columns = {'Total':'Pop_M_2020'}, inplace=True)

# Se unen las tres tablas para formar una sola
df_pop2021 = df_pop2021_T.merge(df_pop2021_M, on='CP').merge(df_pop2021_H, on='CP')
df_pop2021 = df_pop2021[['CP','Tot_Pop_2020','Pop_M_2020','Pop_H_2020']]
df_pop2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 0 to 178
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CP            179 non-null    object
 1   Tot_Pop_2020  179 non-null    object
 2   Pop_M_2020    179 non-null    object
 3   Pop_H_2020    179 non-null    object
dtypes: object(4)
memory usage: 7.0+ KB


In [78]:
df_muni = df_votos_muni.merge(df_pop2021, on='CP', how='left')
df_muni

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat